In [54]:
!pip install tensorflow
import tensorflow as tf
import tensorflow_hub as hub
use_model = hub.load("https://tfhub.dev/google/universal-sentence-encoder/4")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [55]:
import firebase_admin
from firebase_admin import credentials, firestore
import pandas as pd
import numpy as np

In [19]:
cred = credentials.Certificate("/content/serviceAccountKey.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

ValueError: ignored

In [153]:
users = db.collection('users').get()
user_data = []
for doc in users:
    data = doc.to_dict()
    user_data.append(data)

profile_data = []
for doc in users:
    data = doc.to_dict()
    profile_data.append(data['profile'])

In [154]:
user_data = pd.DataFrame(user_data, columns=['uid'])
profile_data = pd.DataFrame(profile_data, columns=['displayName','skills', 'interests'])
merge_data = pd.merge(user_data, profile_data, left_index=True, right_index=True)

result_data = merge_data[['uid', 'displayName', 'skills', 'interests']]
result_data['skills'] = result_data['skills'].apply(lambda skill_list: ', '.join([skill_dict['name'] for skill_dict in skill_list if skill_dict and 'uid' in skill_dict]) if isinstance(skill_list, list) else 'No Skill')
result_data['interests'] = result_data['interests'].apply(lambda interest_list: ', '.join([interest_dict['name'] for interest_dict in interest_list if interest_dict and 'uid' in interest_dict]) if isinstance(interest_list, list) else 'No Interest')

user_data = pd.DataFrame(result_data)
user_data

,uid,displayName,skills,interests
0,06yJpLuZ79Dbzyky0TQL,Daren Lardez,"Go, Usability Testing, IBM Cloud, React Native...",DevOps Engineer
1,0CZfmWDndS6xjVv6o8j3,Carter Piolli,"AWS, Ensemble Methods, React Native, IBM Cloud",Web Analyst
2,0M4QzQ4i5v3jFm0atItg,Alf Chrestien,"TypeScript, Hive, Clustering, vulnerability as...",Big Data Specialist
3,0Ot2WxvdSFpsKEI7ZJGx,Jakie Kleiser,"Java, Naive Bayes, Objective-C, Oracle Cloud",Enterprise System Integration Specialist
4,0RSTq7aC8ygoQxFrBN3A,Torry Mullane,"React, Kafka, Decision Trees, cloud security",API Developer Specialist
...,...,...,...,...
496,ygDFgGFFUk9Ay64ZNNBH,Beitris Forsdicke,"jQuery, Pig, Random Forest, network security",Web Designer
497,ykht9Jt6mCEk1TSpszKX,Yank Dorant,"Scala, Visual Design, DigitalOcean, iOS Develo...",Android Technical developer
498,z7CKDLeIahP4PwgZXYEm,Clarita Liversley,"Kotlin, Prototyping, Red Hat Cloud, Flutter De...",IT Capacity Planning Specialist
499,zGY1ZQzBMWhiPgXwBHTt,Charmain McMurraya,"JavaScript, Storm, Decision Trees, identity an...",Flutter Developer


In [155]:
user_story = []
for index, row in user_data.iterrows():
  user_story.append({
      "uid": row['uid'],
      "story": f"I have Skill {row['skills']}, and I'm interested in {row['interests']}"
  })
user_story

[{'uid': '06yJpLuZ79Dbzyky0TQL',
  'story': "I have Skill Go, Usability Testing, IBM Cloud, React Native Development, and I'm interested in DevOps Engineer"},
 {'uid': '0CZfmWDndS6xjVv6o8j3',
  'story': "I have Skill AWS, Ensemble Methods, React Native, IBM Cloud, and I'm interested in Web Analyst"},
 {'uid': '0M4QzQ4i5v3jFm0atItg',
  'story': "I have Skill TypeScript, Hive, Clustering, vulnerability assessment, and I'm interested in Big Data Specialist"},
 {'uid': '0Ot2WxvdSFpsKEI7ZJGx',
  'story': "I have Skill Java, Naive Bayes, Objective-C, Oracle Cloud, and I'm interested in Enterprise System Integration Specialist"},
 {'uid': '0RSTq7aC8ygoQxFrBN3A',
  'story': "I have Skill React, Kafka, Decision Trees, cloud security, and I'm interested in API Developer Specialist"},
 {'uid': '0S4cUzuqdKtJrnbgGBii',
  'story': "I have Skill Babel, Cassandra, Classification, cloud security, and I'm interested in Android Developer"},
 {'uid': '0b9bUAZukpwoJhYuhs3d',
  'story': "I have Skill jQuery

In [186]:
current_user_id = '06yJpLuZ79Dbzyky0TQL'

current_user = user_data.loc[user_data['uid'] == current_user_id]

current_user = pd.DataFrame(current_user)
current_user

,uid,displayName,skills,interests
0,06yJpLuZ79Dbzyky0TQL,Daren Lardez,"Go, Usability Testing, IBM Cloud, React Native...",DevOps Engineer


In [187]:
# Current user story
current_user_story = f"I have Skill {current_user['skills'][0]} , and I'm interested in {current_user['interests'][0]}"

# Encode the current user story into a vector
current_user_vector = use_model([current_user_story])

current_user_story

"I have Skill Go, Usability Testing, IBM Cloud, React Native Development , and I'm interested in DevOps Engineer"

In [188]:
# Encode all other user stories into vectors and store them in a matrix along with the user IDs
other_user_vectors = []
other_user_ids = []
for user in user_story:
    vector = use_model([user["story"]])
    other_user_vectors.append(vector)
    other_user_ids.append(user["uid"])
other_user_matrix = np.array(other_user_vectors)

In [189]:
# Calculate the similarity scores between the current user vector and all other user vectors in the matrix
similarity_scores = tf.matmul(other_user_matrix, tf.transpose(current_user_vector))

In [191]:
# Sort the similarity scores along with the user IDs in descending order and return the top N most similar users with their IDs
N = 10  # Return the top 10 most similar users
if len(other_user_ids) < N:
    N = len(other_user_ids)
most_similar_users = np.argsort(similarity_scores.numpy().reshape(-1))[::-1][:N]
most_similar_user_ids = [other_user_ids[i] for i in most_similar_users]
most_similar_user_scores = similarity_scores.numpy().reshape(-1)[most_similar_users]

for i in range(1 ,N):
    print(f"User ID: {most_similar_user_ids[i]}, Similarity Score: {most_similar_user_scores[i]}")

User ID: DlmikRZzUud87ZVHWPBv, Similarity Score: 0.8476112484931946
User ID: GYWVAv9MsocnhgVqZHfn, Similarity Score: 0.7643425464630127
User ID: yNY9IhalltJZ02tQU0ix, Similarity Score: 0.7564221024513245
User ID: J5GkwKqxI217nMLNdY1B, Similarity Score: 0.755081057548523
User ID: OtatlU0hDcex4Cn5dRKN, Similarity Score: 0.7529898285865784
User ID: vbxf5J04EfvCznIapW3H, Similarity Score: 0.7501305937767029
User ID: AvufTNycNTN9p5aicfjv, Similarity Score: 0.7341395616531372
User ID: 6mDvGWpmuiCzNoWqkLfP, Similarity Score: 0.734026312828064
User ID: Gt6OGPcCTzzz2waskmn2, Similarity Score: 0.7308772802352905
